Data Extraction

In [1]:
# install scikit learn library
! pip install -U scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB 640.0 kB/s eta 0:00:18
   ---------------------------------------- 0.1/11.1 MB 871.5 kB/s eta 0:00:13
   - -------------------------------------- 0.4/11.1 MB 2.7 MB/s eta 0:00:04
   --- ------------------------------------ 0.9/11.1 MB 4.5 MB/s eta 0:00:03
   ------ --------------------------------- 1.7/11.1 MB 7.3 MB/s eta 0:00:02
   --------- ------------------------------ 2.7/11.1 MB 9.5 MB/s eta 0:00:01
   ------------ --------------------------- 3.4/11.1 MB 10.4 MB/s eta 0:00:01
   -------------- ------------------------- 4.1/11.1 MB 10.9 MB/s eta 0:00:01
   ----------------- ---------------------- 4.8/11.1 MB 11.4 MB/s eta 0:00:01
   -------------------- ------------------- 5.8/11.1 MB 12.2 MB/s eta 0:00:01
   ----------------------- ---------------- 6.5/11.1 MB 12.6 MB/s eta 0:00:01
   -

In [2]:
host = r'127.0.0.1' # denotes that the db in a local installation
db = r'MSDS610' # db we just created
user = r'postgres' # using the postgres user for this demo
pw = r'Ashok%403999' # this is the password established during installation
port = r'5432' # default port estabalished during install
schema = r'raw' # schema we just created

In [4]:
from sqlalchemy import create_engine

# create a connection to the database
engine = create_engine(f'postgresql://{user}:{pw}@{host}:{port}/{db}')
db_conn = engine.connect()

In [7]:
import pandas as pd
table_name = r'Student_Performance'
schema = r'raw' # schema were the data was loaded last week.
Student_raw = pd.read_sql_table(table_name, db_conn, schema)
Student_raw.head()


,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades
0,0.95822,3.4,8.2,53.0,47.0
1,0.85566,3.2,5.9,55.0,35.0
2,0.68025,3.2,9.3,41.0,32.0
3,0.25936,3.2,8.2,47.0,34.0
4,0.60447,3.8,10.0,75.0,33.0


Cleaning 

In [8]:
Student_raw.shape

(1388, 5)

In [9]:
Student_raw.isnull().sum()

Socioeconomic Score    0
Study Hours            0
Sleep Hours            0
Attendance (%)         0
Grades                 0
dtype: int64

In [11]:
# Checking for duplicates
Student_raw.duplicated().sum()

0

In [14]:
Student_raw.Grades.value_counts()

Grades
34.0    349
35.0    194
33.0    167
36.0     47
47.0     33
50.0     31
37.0     30
40.0     30
41.0     28
46.0     28
42.0     27
53.0     26
44.0     26
51.0     26
52.0     25
48.0     25
39.0     25
38.0     24
32.0     24
49.0     23
45.0     23
54.0     21
58.0     16
55.0     14
59.0     13
43.0     13
57.0     13
56.0     12
60.0     11
61.0     10
63.0      9
62.0      9
67.0      7
64.0      7
70.0      4
66.0      3
65.0      3
73.0      2
88.0      1
72.0      1
76.0      1
91.0      1
77.0      1
80.0      1
78.0      1
69.0      1
68.0      1
74.0      1
Name: count, dtype: int64

In [25]:
# Checking the first few rows of your dataset
Student_raw.head()

# Function to categorize numerical columns into categories
def categorize_column(data, thresholds, categories):
    categorized = []
    for value in data:
        for i, threshold in enumerate(thresholds):
            if value < threshold:
                categorized.append(categories[i])
                break
        else:
            categorized.append(categories[-1])  # If value is above all thresholds
    return categorized

# Categories and thresholds for the columns
grades_categories = ['Low', 'Medium', 'High']
grades_thresholds = [40, 60]  # Grades below 40 are Low, between 40-60 are Medium, above 60 are High

attendance_categories = ['Low', 'Medium', 'High']
attendance_thresholds = [50, 75]  # Attendance below 50% is Low, 50-75% is Medium, above 75% is High

socioeconomic_categories = ['Low', 'Medium', 'High']
socioeconomic_thresholds = [0.4, 0.7]  # Socioeconomic score below 0.4 is Low, 0.4-0.7 is Medium, above 0.7 is High

study_hours_categories = ['Low', 'Medium', 'High']
study_hours_thresholds = [3.5, 6.0]  # Study hours below 3.5 is Low, between 3.5-6 is Medium, above 6 is High

sleep_hours_categories = ['Low', 'Medium', 'High']
sleep_hours_thresholds = [6, 8]  # Sleep hours below 6 are Low, 6-8 are Medium, above 8 are High

# Categorizing the relevant columns
Student_raw['Grades_category'] = categorize_column(Student_raw['Grades'], grades_thresholds, grades_categories)
Student_raw['Attendance_category'] = categorize_column(Student_raw['Attendance (%)'], attendance_thresholds, attendance_categories)
Student_raw['Socioeconomic_category'] = categorize_column(Student_raw['Socioeconomic Score'], socioeconomic_thresholds, socioeconomic_categories)
Student_raw['Study_Hours_category'] = categorize_column(Student_raw['Study Hours'], study_hours_thresholds, study_hours_categories)
Student_raw['Sleep_Hours_category'] = categorize_column(Student_raw['Sleep Hours'], sleep_hours_thresholds, sleep_hours_categories)

# Mapping categories to numeric values
def map_to_numeric(dataframe, column_name, categories):
    unique_values = dataframe[column_name].unique()
    mapping_dict = {value: i for i, value in enumerate(categories)}
    dataframe[column_name + '_numeric'] = dataframe[column_name].map(mapping_dict)
    return mapping_dict

# Apply the mapping to the categorized columns
grades_numeric_dict = map_to_numeric(Student_raw, 'Grades_category', grades_categories)
attendance_numeric_dict = map_to_numeric(Student_raw, 'Attendance_category', attendance_categories)
socioeconomic_numeric_dict = map_to_numeric(Student_raw, 'Socioeconomic_category', socioeconomic_categories)
study_hours_numeric_dict = map_to_numeric(Student_raw, 'Study_Hours_category', study_hours_categories)
sleep_hours_numeric_dict = map_to_numeric(Student_raw, 'Sleep_Hours_category', sleep_hours_categories)

# Display the updated DataFrame with categories and numeric columns
Student_raw.head



<bound method NDFrame.head of       Socioeconomic Score  Study Hours  Sleep Hours  Attendance (%)  Grades  \
0                 0.95822          3.4          8.2            53.0    47.0   
1                 0.85566          3.2          5.9            55.0    35.0   
2                 0.68025          3.2          9.3            41.0    32.0   
3                 0.25936          3.2          8.2            47.0    34.0   
4                 0.60447          3.8         10.0            75.0    33.0   
...                   ...          ...          ...             ...     ...   
1383              0.44549          5.5          8.0            51.0    41.0   
1384              0.52466          4.9          6.5            63.0    37.0   
1385              0.88197          3.9          6.2            54.0    36.0   
1386              0.47336          3.5          7.3            61.0    34.0   
1387              0.58119          3.7          9.7            79.0    35.0   

     Grades_category 

In [17]:
Student_raw.head()

,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades,Grades_category,Attendance_category,Socioeconomic_category,Study_Hours_category,Sleep_Hours_category,Grades_category_numeric,Attendance_category_numeric,Socioeconomic_category_numeric,Study_Hours_category_numeric,Sleep_Hours_category_numeric
0,0.95822,3.4,8.2,53.0,47.0,Medium,Medium,High,Low,High,1,1,2,0,2
1,0.85566,3.2,5.9,55.0,35.0,Low,Medium,High,Low,Low,0,1,2,0,0
2,0.68025,3.2,9.3,41.0,32.0,Low,Low,Medium,Low,High,0,0,1,0,2
3,0.25936,3.2,8.2,47.0,34.0,Low,Low,Low,Low,High,0,0,0,0,2
4,0.60447,3.8,10.0,75.0,33.0,Low,High,Medium,Medium,High,0,2,1,1,2


In [19]:
Student_raw['Grades_category'] = categorize_column(Student_raw['Grades'], grades_thresholds, grades_categories)
Student_raw['Grades_category_numeric'] = Student_raw['Grades_category'].map(grades_numeric_dict)
Student_raw['Attendance_category'] = categorize_column(Student_raw['Attendance (%)'], attendance_thresholds, attendance_categories)
Student_raw['Attendance_category_numeric'] = Student_raw['Attendance_category'].map(attendance_numeric_dict)


In [20]:
Student_raw.head()

,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades,Grades_category,Grades_category_numeric,Attendance_category,Attendance_category_numeric,Socioeconomic_category,Socioeconomic_category_numeric,Study_Hours_category,Study_Hours_category_numeric,Sleep_Hours_category,Sleep_Hours_category_numeric
0,0.95822,3.4,8.2,53.0,47.0,Medium,1,Medium,1,High,2,Low,0,High,2
1,0.85566,3.2,5.9,55.0,35.0,Low,0,Medium,1,High,2,Low,0,Low,0
2,0.68025,3.2,9.3,41.0,32.0,Low,0,Low,0,Medium,1,Low,0,High,2
3,0.25936,3.2,8.2,47.0,34.0,Low,0,Low,0,Low,0,Low,0,High,2
4,0.60447,3.8,10.0,75.0,33.0,Low,0,High,2,Medium,1,Medium,1,High,2


Loading Data

In [23]:
import pandas as pd
from sqlalchemy import create_engine


# Inserting the main data into the database
Student_raw.to_sql('student_performance', engine, schema='Cleaned', if_exists='replace', index=False, method='multi', chunksize=1000)

# Example categories (replace these with your actual categories)
grades_categories = [("A", 1), ("B", 2), ("C", 3)]
attendance_categories = [("Excellent", 1), ("Good", 2), ("Fair", 3)]
socioeconomic_categories = [("High", 1), ("Medium", 2), ("Low", 3)]
study_hours_categories = [("High", 1), ("Medium", 2), ("Low", 3)]
sleep_hours_categories = [("High", 1), ("Medium", 2), ("Low", 3)]

# Creating dimension tables from the dictionaries or lists
grades_df = pd.DataFrame(grades_categories, columns=['Grades', 'Grades_id'])
attendance_df = pd.DataFrame(attendance_categories, columns=['Attendance', 'Attendance_id'])
socioeconomic_df = pd.DataFrame(socioeconomic_categories, columns=['Socioeconomic', 'Socioeconomic_id'])
study_hours_df = pd.DataFrame(study_hours_categories, columns=['Study_Hours', 'Study_Hours_id'])
sleep_hours_df = pd.DataFrame(sleep_hours_categories, columns=['Sleep_Hours', 'Sleep_Hours_id'])

# Inserting the dimension tables into the database
grades_df.to_sql('grades', engine, schema='Cleaned', if_exists='replace', index=False)
attendance_df.to_sql('attendance', engine, schema='Cleaned', if_exists='replace', index=False)
socioeconomic_df.to_sql('socioeconomic', engine, schema='Cleaned', if_exists='replace', index=False)
study_hours_df.to_sql('study_hours', engine, schema='Cleaned', if_exists='replace', index=False)
sleep_hours_df.to_sql('sleep_hours', engine, schema='Cleaned', if_exists='replace', index=False)



3